In [1]:
import pandas as pd 
import requests
import time
from bs4 import BeautifulSoup

Scraping data from The Number website

In [2]:
# functions where you put the url of the page of "the-numbers" and give you back dataframe out of it
def scrape_the_number(url):
    headers = {"User-Agent": "Mozilla/5.0"}
    rcs = requests.get(url, headers=headers)
    soup = BeautifulSoup(rcs.text, "html.parser")
    rows = soup.select("table tr")
    data = []

    for row in rows:
        cells = row.find_all("td")
        if cells:
            data.append([cell.text.strip() for cell in cells])

    df = pd.DataFrame(data, columns=["Index", "Release Date", "Title", "Production Budget", "Domestic Gross", "Worldwide Gross"])
    return df

In [ ]:
#applying the dunction and getting 1000 rows
page1 = scrape_the_number("https://www.the-numbers.com/movie/budgets/all")
page2 = scrape_the_number("https://www.the-numbers.com/movie/budgets/all/101")
page3 = scrape_the_number("https://www.the-numbers.com/movie/budgets/all/201")
page4 = scrape_the_number("https://www.the-numbers.com/movie/budgets/all/301")
page5 = scrape_the_number("https://www.the-numbers.com/movie/budgets/all/401")
page6 = scrape_the_number("https://www.the-numbers.com/movie/budgets/all/501")
page7 = scrape_the_number("https://www.the-numbers.com/movie/budgets/all/601")
page8 = scrape_the_number("https://www.the-numbers.com/movie/budgets/all/701")
page9 = scrape_the_number("https://www.the-numbers.com/movie/budgets/all/801")
page10 = scrape_the_number("https://www.the-numbers.com/movie/budgets/all/901")


Concating the pages to one result

In [4]:
the_number_df = pd.concat([page1, page2, page3, page4, page5, page6, page7, page8, page9, page10], ignore_index=True)
the_number_df =  the_number_df.drop(columns="Index")

Now using the OMDB api though a function 

In [ ]:
# 2. OMDB function to apply on the list by title to get info

key = "2f9babf3"
def get_movie_info(title):
    url = f"http://www.omdbapi.com/?t={title}&apikey={key}"
    res = requests.get(url).json()
    time.sleep(0.2)
    
    return res

omdb_data = the_number_df["Title"].apply(get_movie_info)

In [53]:
# converting the results to pd dataframe
omdb_df = pd.DataFrame(omdb_data.tolist())

adding one column with made index to match the two dataffram (the two dataframe has the same order but they don't necceserlly match the title)

In [ ]:
omdb = omdb_df.reset_index()
the_number = the_number_df.reset_index()

Merging the two dataframes

In [66]:
df_full_col = pd.merge(the_number, omdb, on = "index", how = "inner")

Saving the data to my local disk

In [1]:
# this would save it in the folder that you are working on
df_full_col.to_csv("df_full_col_muayad.csv", index=False)

NameError: name 'df_full_col' is not defined

Cleaning data

In [76]:
df_full_col.head()

,index,Release Date,Title_x,Production Budget,Domestic Gross,Worldwide Gross,Response,Error,Title_y,Year,...,Metascore,imdbRating,imdbVotes,imdbID,Type,DVD,BoxOffice,Production,Website,totalSeasons
0,0,"Dec 16, 2015",Star Wars Ep. VII: The Force Awakens,"$533,200,000","$936,662,225","$2,056,046,835",False,Movie not found!,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,"Apr 23, 2019",Avengers: Endgame,"$400,000,000","$858,373,000","$2,717,503,922",True,NaN,Avengers: Endgame,2019,...,78,8.4,"1,390,337",tt4154796,movie,N/A,"$858,373,000",N/A,N/A,NaN
2,2,"Dec 9, 2022",Avatar: The Way of Water,"$400,000,000","$688,809,501","$2,322,902,023",True,NaN,Avatar: The Way of Water,2022,...,67,7.5,"546,561",tt1630029,movie,N/A,"$684,075,767",N/A,N/A,NaN
3,3,"May 17, 2025",Mission: Impossible—The Final Reckoning,"$400,000,000","$197,413,515","$591,353,074",True,NaN,Mission: Impossible - The Final Reckoning,2025,...,67,7.3,"160,935",tt9603208,movie,N/A,"$197,413,515",N/A,N/A,NaN
4,4,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$379,000,000","$241,071,802","$1,045,713,802",True,NaN,Pirates of the Caribbean: On Stranger Tides,2011,...,45,6.6,"594,656",tt1298650,movie,N/A,"$241,071,802",N/A,N/A,NaN
